This Notebook runs simple one-dimensional FUNWAVE simulations. 
-----

It consists of the following four steps:

* **Step #1** - Bathymetry: In this step you can generate the one-dimensional bathymetry that will be used in the FUNWAVE simulation. The user has the option of creating a flat bathymetry, a slope bathymetry or upload his own file. Once the depth file is generated, proceed to Step #2.

* **Step #2** - Input File: In this tab you can generate the Input file with your desired specifications for the simulation. After generating your input file, proceed to Step #3.

* **Step #3** - Run Funwave: This tab lets you run FUNWAVE without the need of a terminal. You can also see the simulation's proggress and abort it if it is necessary. Proceed to Step #4 to process your results.

* **Step #4** - Post Processing: This tab gives you the option of generating an image at an specific time or generate a video of the simulation.

In [1]:
##### %matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# import project title and generate project button from PRINCIPAL_TAB 
# once the button is pressed, the entire GUI will appear
# the function that does this is located at wavemaker_Function.py
from pyFiles.PRINCIPAL_TAB import container_title,space_box2, project_button

display(space_box2,container_title,space_box2) # display project text box and button

############################        
#### Activate Functions ####
############################

from pyFiles.wavemaker_Function import project_clicked # import generate project project fucntions

from pyFiles.bathymetry_Functions import compute_high_difference, on_save_clicked, on_plot_clicked
# ^ import generate bathymetry functions


from pyFiles.input_Functions import update_input_function # activate update input values button
from pyFiles.input_Functions import boolean_function # activate generate input file button

# activate the output variables (located in PrincipalTab_3) that will be shown the post-process list:
from pyFiles.output_functions import toggle_output_ETA
from pyFiles.output_functions import toggle_output_Hmax
from pyFiles.output_functions import toggle_output_Hmin
from pyFiles.output_functions import toggle_output_ETAmean
from pyFiles.output_functions import toggle_choose_output


from pyFiles.video_Functions import runVID_function,save_plot_clicked, plot_output_clicked
# ^ activate post processing buttons

from pyFiles.funwave_Functions import runFUN_function # activate run funwave button


VBox(children=(Box(layout=Layout(height='20px', width='90%')), HBox(children=(HTML(value='Project Title:'), Text(value='WK2', layout=Layout(width='40%')), Box(layout=Layout(width='10%')), Button(description='Generate Project', layout=Layout(height='40px', width='30%'), style=ButtonStyle())), layout=Layout(width='90%')), Box(layout=Layout(height='20px', width='90%')), Tab(children=(VBox(children=(VBox(children=(HTML(value='Specify the type of One-Dimensional Bathymetry:', layout=Layout(height='25px')), Dropdown(options=('Select Bathy', 'Upload File', 'Slope', 'Flat'), value='Select Bathy')), layout=Layout(height='70px')), VBox(children=(VBox(children=(HTML(value='Upload your file in the <b>Project Title</b> folder. Once the file is uploaded, identify \nits name (e.g. MyBathy.txt) and its total horizontal length (THL). Press "Plot Bathymetry" to visualize the bathymetry.<br><br>\n<b>NOTE: </b>This file must be a text of 1 row; with depth values [-] for underwater and [+] for surface. <br>Also, the values must be in <b>metric</b> units.', layout=Layout(width='90%')), Box(layout=Layout(height='25px')), HBox(children=(HTML(value='File name:', layout=Layout(width='10%')), Text(value='', layout=Layout(width='25%')), BoundedFloatText(value=200.0, description='THL', layout=Layout(width='25%'), max=1500.0, min=10.0, step=0.01)), layout=Layout(height='100px')), Box(layout=Layout(height='25px')), HTML(value='Make sure to press <b>"Assemble Bathymetry File"</b> before continuing to <b>Step #2</b>. This will format the uploaded bathymetry file to the <b>required FUNWAVE format.</b>', layout=Layout(width='90%'))), layout=Layout(display='none', height='500px')), VBox(children=(HTML(value='This Option plots and saves simple one-dimensional <b>slope</b>\nbathymetries consisting of 2 segments and 3 vertices. The user can select the <b>total horizontal lenght</b> \nof the bathymetry <b>(THL)</b>, the <b>spacing</b> between the points <b>(DX)</b>,\nand the <b>elevation</b> and <b>location</b> of the vertices. Once the desired bathymetry is plotted,\npress the "Assemble Bathymetry File"\nbutton and proceed to Step #2.<br>The values are in <b>metric</b> units.', layout=Layout(width='90%')), Box(layout=Layout(height='25px')), HBox(children=(BoundedFloatText(value=200.0, description='THL', layout=Layout(width='25%'), max=1500.0, min=10.0, step=0.01), BoundedFloatText(value=1.0, description='DX', layout=Layout(width='25%'), step=0.5)), layout=Layout(align_items='center', height='40px', width='90%')), Box(layout=Layout(height='25px')), HTML(value='<b>NOTE:</b> Depth values are [-] for underwater and [+] for surface.'), HBox(children=(VBox(children=(HTML(value='<b>Vertex #1<b/>'), FloatSlider(value=-10.0, description='Elevation ', layout=Layout(height='50px', width='100%'), max=15.0, min=-10.0, step=0.01), HTML(value='Located at <b>0.0 meters</b>.')), layout=Layout(align_items='center', border='solid 2px grey', display='flex-grow', width='30%')), VBox(children=(HTML(value='<b>Vertex #2<b/>'), FloatSlider(value=-10.0, description='Elevation ', layout=Layout(height='50px', width='100%'), max=15.0, min=-10.0, step=0.01), FloatSlider(value=10.0, description='Location', layout=Layout(height='50px', width='100%'), max=200.0, step=0.01)), layout=Layout(align_items='center', border='solid 2px grey', display='flex-grow', width='30%')), VBox(children=(HTML(value='<b>Vertex #3<b/>'), FloatSlider(value=-10.0, description='Elevation', layout=Layout(height='50px', width='100%'), max=15.0, min=-10.0, step=0.01), HTML(value='Located at <b>THL</b> distance.')), layout=Layout(align_items='center', border='solid 2px grey', display='flex-grow', width='30%')))), Box(layout=Layout(height='25px'))), layout=Layout(display='none')), VBox(children=(HTML(value='This Option plots and saves simple one-dimensional <b>flat</b>\nbathymetries with a constant depth. The user can select the <b>total horizontal lenght</b> \nof the bathymetry <b>(THL)</b>, the <b>spacing</b> between the poi

In [16]:
totalTime = 300
plotInt_text = 10
steady_time = 100
T_INTV_MEAN = 50

TimeBegin_text1 = 20
TimeLimit_text1 = 400

TimeBegin_text2 = 150
TimeLimit_text2 = 250 

files1_start = [(TimeBegin_text1/plotInt_text) + 1] # number of files for eta, hmin & hmax
files2_start = [(TimeBegin_text2/T_INTV_MEAN)-(steady_time/T_INTV_MEAN)] # number of files for etaMean & wave height

files1_end = [(TimeLimit_text1/plotInt_text) + 1] # number of files for eta, hmin & hmax
files2_end = [(TimeLimit_text2/T_INTV_MEAN)-(steady_time/T_INTV_MEAN)] # number of files for etaMean & wave height

In [19]:
for num in range(int(files1_start[0]),int(files1_end[0])+1):
    fnum = '%.5d' % (num)
    print('fnum = %s' % (fnum))
    
    NUM = num-int(files1_start[0])+1
    if NUM < 10:
        fileIndex = '0'+str(NUM)
        print('fileIndex = ',fileIndex)
    elif NUM < 100:
        fileIndex = str(NUM)
        print('fileIndex = ',fileIndex)
    elif NUM < 1000: 
        fileIndex = str(NUM)
        print('fileIndex = ',fileIndex)
    else: 
        fileIndex = str(NUM)
        print('fileIndex = ',fileIndex)

fnum = 00003
fileIndex =  01
fnum = 00004
fileIndex =  02
fnum = 00005
fileIndex =  03
fnum = 00006
fileIndex =  04
fnum = 00007
fileIndex =  05
fnum = 00008
fileIndex =  06
fnum = 00009
fileIndex =  07
fnum = 00010
fileIndex =  08
fnum = 00011
fileIndex =  09
fnum = 00012
fileIndex =  10
fnum = 00013
fileIndex =  11
fnum = 00014
fileIndex =  12
fnum = 00015
fileIndex =  13
fnum = 00016
fileIndex =  14
fnum = 00017
fileIndex =  15
fnum = 00018
fileIndex =  16
fnum = 00019
fileIndex =  17
fnum = 00020
fileIndex =  18
fnum = 00021
fileIndex =  19
fnum = 00022
fileIndex =  20
fnum = 00023
fileIndex =  21
fnum = 00024
fileIndex =  22
fnum = 00025
fileIndex =  23
fnum = 00026
fileIndex =  24
fnum = 00027
fileIndex =  25
fnum = 00028
fileIndex =  26
fnum = 00029
fileIndex =  27
fnum = 00030
fileIndex =  28
fnum = 00031
fileIndex =  29
fnum = 00032
fileIndex =  30
fnum = 00033
fileIndex =  31
fnum = 00034
fileIndex =  32
fnum = 00035
fileIndex =  33
fnum = 00036
fileIndex =  34
fnum = 00037
f

In [9]:
files1_start

[3.0]